In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.callbacks import StreamingStdOutCallbackHandler

# 대화 전체를 저장하는 메모리
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input":"hi"}, {"output":"How are you"})
memory.load_memory_variables({})





In [ ]:
from langchain.memory import ConversationBufferWindowMemory


#최근 대화를 저장하는 메모리
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

add_message("테스트인풋1", "테스트아웃풋1")
add_message("테스트인풋2", "테스트아웃풋2")
add_message("테스트인풋3", "테스트아웃풋3")
add_message("테스트인풋4", "테스트아웃풋4")
add_message("테스트인풋5", "테스트아웃풋5")
add_message("테스트인풋6", "테스트아웃풋6")

memory.load_memory_variables({})



In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})
    
def get_history():
    return memory.load_memory_variables({})
    
add_message("난 이원국이고, 대한민국 서울에 살아", "와우! 멋져!")


In [ ]:
add_message("한국은 매우 아름다워", "나도 가보고 싶어")


In [ ]:
get_history()

In [87]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True
    )

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("난 이원국이고, 대한민국 서울에 살아", "와우! 멋져!")
get_history()

{'history': [HumanMessage(content='난 이원국이고, 대한민국 서울에 살아'),
  AIMessage(content='와우! 멋져!')]}